Adversarial training steps
1. Train model on original inputs (at very beginning) for N_1 iterations to obtain the gradient
2. Run model again for N_2 iterations, but this time sample from X_train and apply FGSM
3. Store that as the actual model

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.utils import np_utils
import keras
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
import os
import pickle
%matplotlib inline

Using TensorFlow backend.


In [2]:
# load MNIST

## Loading MNIST dataset from keras

def load_mnist(path='MNIST.pickle'):
    with open(os.path.expanduser(path), 'rb') as f:
        train, val, test = pickle.load(f, encoding='latin1')
#         del save  # hint to help gc free up memory

        return train, val, test



train, val, test = load_mnist("data/mnist.pkl")
X_train, Y_train, X_val, Y_val, X_test, Y_test = train[0], train[1], val[0], val[1], test[0], test[1]

## Changing dimension of input images from N*28*28 to  N*784
# X_train = X_train.reshape((X_train.shape[0],X_train.shape[1]*X_train.shape[2]))
# X_test = X_test.reshape((X_test.shape[0],X_test.shape[1]*X_test.shape[2]))
# print('Train dimension:');print(X_train.shape)
# print('Test dimension:');print(X_test.shape)
## Changing labels to one-hot encoded vector
lb = LabelBinarizer()
Y_train = lb.fit_transform(Y_train)
Y_test = lb.transform(Y_test)
print('Train labels dimension:');print(Y_train.shape)
print('Test labels dimension:');print(Y_test.shape)

Train labels dimension:
(50000, 10)
Test labels dimension:
(10000, 10)


In [3]:
# original model

# https://towardsdatascience.com/multi-layer-perceptron-using-tensorflow-9f3e218a4809

print("With Dropout & Batch Normalization")

import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score

from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
# Batch normalization implementation
# from https://github.com/tensorflow/tensorflow/issues/1122
def batch_norm_layer(inputT, is_training=True, scope=None):
    # Note: is_training is tf.placeholder(tf.bool) type
    return tf.cond(is_training,
                    lambda: batch_norm(inputT, is_training=True,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope=scope),
                    lambda: batch_norm(inputT, is_training=False,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9,
                    scope=scope, reuse = True))


s = tf.InteractiveSession()

## Defining various initialization parameters for 784-512-256-10 MLP model
num_classes = Y_train.shape[1]
num_features = X_train.shape[1]
num_output = Y_train.shape[1]
num_layers_0 = 200
num_layers_1 = 200
num_layers_2 = 200
starter_learning_rate = 0.001
regularizer_rate = 0.1

# Placeholders for the input data
input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
input_Y = tf.placeholder('float32',shape = (None,num_classes),name='input_Y')



## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
weights_0 = tf.Variable(tf.random_normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
bias_0 = tf.Variable(tf.random_normal([num_layers_0]))
weights_1 = tf.Variable(tf.random_normal([num_layers_0,num_layers_1], stddev=(1/tf.sqrt(float(num_layers_0)))))
bias_1 = tf.Variable(tf.random_normal([num_layers_1]))
weights_2 = tf.Variable(tf.random_normal([num_layers_1,num_layers_2], stddev=(1/tf.sqrt(float(num_layers_1)))))
bias_2 = tf.Variable(tf.random_normal([num_layers_2]))
weights_3 = tf.Variable(tf.random_normal([num_layers_2,num_output], stddev=(1/tf.sqrt(float(num_layers_2)))))
bias_3 = tf.Variable(tf.random_normal([num_output]))

# for dropout layer
keep_prob = tf.placeholder(tf.float32)
# Initializing weigths and biases -- with dropout
h0 = tf.matmul(input_X,weights_0)+bias_0
batch_mean0, batch_var0 = tf.nn.moments(h0,[0])
hidden_output_0 = tf.nn.relu(tf.nn.batch_normalization(h0, batch_mean0, batch_var0, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_0_0 = tf.nn.dropout(hidden_output_0, keep_prob)
h1 = tf.matmul(hidden_output_0,weights_1)+bias_1
batch_mean1, batch_var1 = tf.nn.moments(h1,[0])
hidden_output_1 = tf.nn.relu(tf.nn.batch_normalization(h1, batch_mean1, batch_var1, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_1_1 = tf.nn.dropout(hidden_output_1, keep_prob)
h2 = tf.matmul(hidden_output_1,weights_2)+bias_2
batch_mean2, batch_var2 = tf.nn.moments(h2,[0])
hidden_output_2 = tf.nn.relu(tf.nn.batch_normalization(h2, batch_mean2, batch_var2, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_2_2 = tf.nn.dropout(hidden_output_2, keep_prob)
predicted_Y = tf.sigmoid(tf.matmul(hidden_output_2_2,weights_3) + bias_3)



# ## Initializing weigths and biases -- withOUT dropout
# hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0)+bias_0)
# hidden_output_1 = tf.nn.relu(tf.matmul(hidden_output_0,weights_1)+bias_1)
# hidden_output_2 = tf.nn.relu(tf.matmul(hidden_output_1,weights_2)+bias_2)
# predicted_Y = tf.sigmoid(tf.matmul(hidden_output_2,weights_3) + bias_3)

## Defining the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predicted_Y,labels=input_Y)) \
        + regularizer_rate*(tf.reduce_sum(tf.square(bias_0)) + tf.reduce_sum(tf.square(bias_1)) + tf.reduce_sum(tf.square(bias_2)))

## Variable learning rate
learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
## Adam optimzer for finding the right weight
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,weights_2,weights_3,
                                                                         bias_0,bias_1,bias_2,bias_3])
grads_wrt_input_tensor = tf.gradients(loss,input_X)[0]
# preoptimizer = tf.train.AdamOptimizer(learning_rate)
# # grads = preoptimizer.compute_gradients(input_Y)
# grads = preoptimizer.compute_gradients(loss)
# grad_placeholder = [(tf.placeholder("float", shape=grad[1].get_shape()), grad[1]) for grad in grads]
# optimizer = preoptimizer.minimize(loss,var_list=[weights_0,weights_1,weights_2,weights_3,bias_0,bias_1,bias_2,bias_3])

## Metrics definition
correct_prediction = tf.equal(tf.argmax(Y_train,1), tf.argmax(predicted_Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Training parameters
batch_size = 128
epochs=14
dropout_prob = 0.1
training_accuracy = []
training_loss = []
testing_accuracy = []
s.run(tf.global_variables_initializer())
for epoch in range(epochs):    
    arr = np.arange(X_train.shape[0])
    np.random.shuffle(arr)
    for index in range(0,X_train.shape[0],batch_size):
        _, grads_wrt_input = s.run([optimizer, grads_wrt_input_tensor], {input_X: X_train[arr[index:index+batch_size]],
                          input_Y: Y_train[arr[index:index+batch_size]],
                        keep_prob:dropout_prob})
        
#         vars_with_grads = s.run(grads, feed_dict={input_X: X_train[arr[index:index+batch_size]],
#                           input_Y: Y_train[arr[index:index+batch_size]],
#                         keep_prob:dropout_prob})
    
    training_accuracy.append(s.run(accuracy, feed_dict= {input_X:X_train, 
                                                         input_Y: Y_train,keep_prob:1}))
    training_loss.append(s.run(loss, {input_X: X_train, 
                                      input_Y: Y_train,keep_prob:1}))
    
    ## Evaluation of model
    testing_accuracy.append(accuracy_score(Y_test.argmax(1), 
                            s.run(predicted_Y, {input_X: X_test,keep_prob:1}).argmax(1)))
#     testing_accuracy.append(accuracy_score(Y_test.argmax(1), 
#                             s.run(predicted_Y, {input_X: X_test}).argmax(1)))
    print("Epoch:{0} | Train loss: {1:.2f} | Train acc: {2:.3f} | Test acc:{3:.3f}".format(epoch,
                                                                    training_loss[epoch],
                                                                    training_accuracy[epoch],
                                                                   testing_accuracy[epoch]))
# grad_vals = s.run([grad[0] for grad in grads])
# https://r2rt.com/implementing-batch-normalization-in-tensorflow.html

With Dropout & Batch Normalization


W0210 16:15:34.069361 139952431105792 deprecation.py:506] From <ipython-input-3-cc389274d984>:57: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Epoch:0 | Train loss: 33.31 | Train acc: 0.948 | Test acc:0.948
Epoch:1 | Train loss: 18.70 | Train acc: 0.965 | Test acc:0.960
Epoch:2 | Train loss: 10.71 | Train acc: 0.973 | Test acc:0.968
Epoch:3 | Train loss: 6.34 | Train acc: 0.979 | Test acc:0.974
Epoch:4 | Train loss: 3.98 | Train acc: 0.983 | Test acc:0.974
Epoch:5 | Train loss: 2.72 | Train acc: 0.985 | Test acc:0.978
Epoch:6 | Train loss: 2.07 | Train acc: 0.985 | Test acc:0.975
Epoch:7 | Train loss: 1.75 | Train acc: 0.986 | Test acc:0.975
Epoch:8 | Train loss: 1.59 | Train acc: 0.989 | Test acc:0.979
Epoch:9 | Train loss: 1.52 | Train acc: 0.989 | Test acc:0.978
Epoch:10 | Train loss: 1.49 | Train acc: 0.991 | Test acc:0.977
Epoch:11 | Train loss: 1.48 | Train acc: 0.992 | Test acc:0.980
Epoch:12 | Train loss: 1.47 | Train acc: 0.992 | Test acc:0.978
Epoch:13 | Train loss: 1.47 | Train acc: 0.992 | Test acc:0.982


In [4]:
grads_wrt_input.shape
# https://stackoverflow.com/questions/36245481/tensorflow-slow-performance-when-getting-gradients-at-inputs

# This gradient output tells us the gradient to be applied for every position of a tensor of MNIST inputs
# one MNIST x array has 784 values, and this is the gradient for each of the 784 values

(80, 784)

In [46]:
prop_random = 0.2 # what proportion of the training set do you want to apply perturbations to (!= sampling!!!)
# Might not be wise to shuffle the dataset, given that we kept X and Y separate -- let's just generate random index to start from
import random
# last_index_to_sample_from = int(X_train.shape[0] - X_train.shape[0]*prop_random)
# start_index = random.randint(0,last_index_to_sample_from)
# end_index = start_index + int(X_train.shape[0]*prop_random)
# print("Sample index range: ", (start_index, end_index))

ind = np.arange(int(X_train.shape[0]))
np.random.shuffle(ind)#[0:int(X_train.shape[0]*prop_random)]
ind = ind[0:int(X_train.shape[0]*prop_random)]

# sample from training set, matrix addition
# sample_X_train = X_train[start_index,end_index] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
# sample_Y_train = Y_train[start_index,end_index]

sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
sample_Y_train = Y_train[[ind]]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [49]:
grads_wrt_input[-1] # use last row for gradients?

sample_X_train.shape

(10000, 784)

In [65]:
import keras.backend as K
# from attack_utils import gen_grad

def fgsm(x, grad, eps=0.3, clipping=True):
    """
    FGSM attack.
    """
    # signed gradient
    normed_grad = K.sign(grad).eval()

    # Multiply by constant epsilon
    scaled_grad = eps * normed_grad

    # Add perturbation to original example to obtain adversarial example
    adv_x = K.stop_gradient(x + scaled_grad)

    if clipping:
        adv_x = K.clip(adv_x, 0, 1)
    return adv_x

# x_adv = fgsm(sample_X_train, grads_wrt_input[-1], eps=16/255, clipping=True).eval()


In [67]:
# K.sign(grads_wrt_input[-1]).eval()
# x_adv

In [89]:
# execute adversarial training --> compile into single pipeline to be turned into SOLID-able pipeline component

# original model

# https://towardsdatascience.com/multi-layer-perceptron-using-tensorflow-9f3e218a4809

print("With Dropout & Batch Normalization & Adversarial Training")

import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score

from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
# Batch normalization implementation
# from https://github.com/tensorflow/tensorflow/issues/1122
def batch_norm_layer(inputT, is_training=True, scope=None):
    # Note: is_training is tf.placeholder(tf.bool) type
    return tf.cond(is_training,
                    lambda: batch_norm(inputT, is_training=True,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope=scope),
                    lambda: batch_norm(inputT, is_training=False,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9,
                    scope=scope, reuse = True))


s = tf.InteractiveSession()

prop_random = 0.2 # what proportion of the training set do you want to apply perturbations to (!= sampling!!!)
adv_placeholders = tf.zeros([int(Y_train.shape[0]*prop_random) * Y_train.shape[1]]).eval().reshape(int(Y_train.shape[0]*prop_random), Y_train.shape[1])
modified_Y_train = np.array([list(Y_train)+list(adv_placeholders)][0])

## Defining various initialization parameters for 784-512-256-10 MLP model
num_classes = Y_train.shape[1]
num_features = X_train.shape[1]
num_output = Y_train.shape[1]
num_layers_0 = 200
num_layers_1 = 200
num_layers_2 = 200
starter_learning_rate = 0.001
regularizer_rate = 0.1

# Placeholders for the input data
input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
input_Y = tf.placeholder('float32',shape = (None,num_classes),name='input_Y')



## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
weights_0 = tf.Variable(tf.random_normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
bias_0 = tf.Variable(tf.random_normal([num_layers_0]))
weights_1 = tf.Variable(tf.random_normal([num_layers_0,num_layers_1], stddev=(1/tf.sqrt(float(num_layers_0)))))
bias_1 = tf.Variable(tf.random_normal([num_layers_1]))
weights_2 = tf.Variable(tf.random_normal([num_layers_1,num_layers_2], stddev=(1/tf.sqrt(float(num_layers_1)))))
bias_2 = tf.Variable(tf.random_normal([num_layers_2]))
weights_3 = tf.Variable(tf.random_normal([num_layers_2,num_output], stddev=(1/tf.sqrt(float(num_layers_2)))))
bias_3 = tf.Variable(tf.random_normal([num_output]))

# for dropout layer
keep_prob = tf.placeholder(tf.float32)
# Initializing weigths and biases -- with dropout
h0 = tf.matmul(input_X,weights_0)+bias_0
batch_mean0, batch_var0 = tf.nn.moments(h0,[0])
hidden_output_0 = tf.nn.relu(tf.nn.batch_normalization(h0, batch_mean0, batch_var0, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_0_0 = tf.nn.dropout(hidden_output_0, keep_prob)
h1 = tf.matmul(hidden_output_0,weights_1)+bias_1
batch_mean1, batch_var1 = tf.nn.moments(h1,[0])
hidden_output_1 = tf.nn.relu(tf.nn.batch_normalization(h1, batch_mean1, batch_var1, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_1_1 = tf.nn.dropout(hidden_output_1, keep_prob)
h2 = tf.matmul(hidden_output_1,weights_2)+bias_2
batch_mean2, batch_var2 = tf.nn.moments(h2,[0])
hidden_output_2 = tf.nn.relu(tf.nn.batch_normalization(h2, batch_mean2, batch_var2, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_2_2 = tf.nn.dropout(hidden_output_2, keep_prob)
predicted_Y = tf.sigmoid(tf.matmul(hidden_output_2_2,weights_3) + bias_3)



# ## Initializing weigths and biases -- withOUT dropout
# hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0)+bias_0)
# hidden_output_1 = tf.nn.relu(tf.matmul(hidden_output_0,weights_1)+bias_1)
# hidden_output_2 = tf.nn.relu(tf.matmul(hidden_output_1,weights_2)+bias_2)
# predicted_Y = tf.sigmoid(tf.matmul(hidden_output_2,weights_3) + bias_3)

## Defining the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predicted_Y,labels=input_Y)) \
        + regularizer_rate*(tf.reduce_sum(tf.square(bias_0)) + tf.reduce_sum(tf.square(bias_1)) + tf.reduce_sum(tf.square(bias_2)))

## Variable learning rate
learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
## Adam optimzer for finding the right weight
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,weights_2,weights_3,
                                                                         bias_0,bias_1,bias_2,bias_3])
# grads_wrt_input_tensor = tf.gradients(loss,input_X)[0]
# preoptimizer = tf.train.AdamOptimizer(learning_rate)
# # grads = preoptimizer.compute_gradients(input_Y)
# grads = preoptimizer.compute_gradients(loss)
# grad_placeholder = [(tf.placeholder("float", shape=grad[1].get_shape()), grad[1]) for grad in grads]
# optimizer = preoptimizer.minimize(loss,var_list=[weights_0,weights_1,weights_2,weights_3,bias_0,bias_1,bias_2,bias_3])

## Metrics definition
correct_prediction = tf.equal(tf.argmax(modified_Y_train,1), tf.argmax(predicted_Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Training parameters
batch_size = 128
epochs=14
dropout_prob = 0.1
training_accuracy = []
training_loss = []
testing_accuracy = []
s.run(tf.global_variables_initializer())
for epoch in range(epochs):    
    
    # regenerate training set
    # Might not be wise to shuffle the dataset, given that we kept X and Y separate -- let's just generate random index to start from
    import random

    ind = np.arange(int(X_train.shape[0]))
    np.random.shuffle(ind)#[0:int(X_train.shape[0]*prop_random)]
    ind = ind[0:int(X_train.shape[0]*prop_random)]

    sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
    sample_Y_train = Y_train[[ind]]
    x_adv = fgsm(sample_X_train, grads_wrt_input[-1], eps=16/255, clipping=True).eval()
    adv_combined_X_train = np.array([list(X_train)+list(x_adv)][0])
    adv_combined_Y_train = np.array([list(Y_train)+list(sample_Y_train)][0])
    
    
    arr = np.arange(adv_combined_X_train.shape[0])
    np.random.shuffle(arr)
    for index in range(0,adv_combined_X_train.shape[0],batch_size):
        s.run(optimizer, {input_X: adv_combined_X_train[arr[index:index+batch_size]],
                          input_Y: adv_combined_Y_train[arr[index:index+batch_size]],
                        keep_prob:dropout_prob})
        
#         vars_with_grads = s.run(grads, feed_dict={input_X: X_train[arr[index:index+batch_size]],
#                           input_Y: Y_train[arr[index:index+batch_size]],
#                         keep_prob:dropout_prob})
    
    training_accuracy.append(s.run(accuracy, feed_dict= {input_X:adv_combined_X_train, 
                                                         input_Y: adv_combined_Y_train,keep_prob:1}))
    training_loss.append(s.run(loss, {input_X: adv_combined_X_train, 
                                      input_Y: adv_combined_Y_train,keep_prob:1}))
    
    ## Evaluation of model
    testing_accuracy.append(accuracy_score(Y_test.argmax(1), 
                            s.run(predicted_Y, {input_X: X_test,keep_prob:1}).argmax(1)))
#     testing_accuracy.append(accuracy_score(Y_test.argmax(1), 
#                             s.run(predicted_Y, {input_X: X_test}).argmax(1)))
    print("Epoch:{0} | Train loss: {1:.2f} | Train acc: {2:.3f} | Test acc:{3:.3f}".format(epoch,
                                                                    training_loss[epoch],
                                                                    training_accuracy[epoch],
                                                                   testing_accuracy[epoch]))
# grad_vals = s.run([grad[0] for grad in grads])
# https://r2rt.com/implementing-batch-normalization-in-tensorflow.html

With Dropout & Batch Normalization & Adversarial Training


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:120: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:121: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result

Epoch:0 | Train loss: 32.62 | Train acc: 0.810 | Test acc:0.952
Epoch:1 | Train loss: 16.77 | Train acc: 0.824 | Test acc:0.965
Epoch:2 | Train loss: 8.79 | Train acc: 0.830 | Test acc:0.970
Epoch:3 | Train loss: 4.83 | Train acc: 0.833 | Test acc:0.971
Epoch:4 | Train loss: 2.94 | Train acc: 0.837 | Test acc:0.975
Epoch:5 | Train loss: 2.09 | Train acc: 0.838 | Test acc:0.979
Epoch:6 | Train loss: 1.72 | Train acc: 0.840 | Test acc:0.979
Epoch:7 | Train loss: 1.57 | Train acc: 0.841 | Test acc:0.978
Epoch:8 | Train loss: 1.51 | Train acc: 0.841 | Test acc:0.977
Epoch:9 | Train loss: 1.48 | Train acc: 0.842 | Test acc:0.978
Epoch:10 | Train loss: 1.47 | Train acc: 0.842 | Test acc:0.980
Epoch:11 | Train loss: 1.47 | Train acc: 0.843 | Test acc:0.979
Epoch:12 | Train loss: 1.47 | Train acc: 0.843 | Test acc:0.980
Epoch:13 | Train loss: 1.47 | Train acc: 0.844 | Test acc:0.980


# Complete adversarial training pipeline

In [90]:
##########################################################################################################
################################## GRADIENTS #############################################################
##########################################################################################################

# https://towardsdatascience.com/multi-layer-perceptron-using-tensorflow-9f3e218a4809

print("With Dropout & Batch Normalization")

import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score

from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
# Batch normalization implementation
# from https://github.com/tensorflow/tensorflow/issues/1122
def batch_norm_layer(inputT, is_training=True, scope=None):
    # Note: is_training is tf.placeholder(tf.bool) type
    return tf.cond(is_training,
                    lambda: batch_norm(inputT, is_training=True,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope=scope),
                    lambda: batch_norm(inputT, is_training=False,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9,
                    scope=scope, reuse = True))


s = tf.InteractiveSession()

## Defining various initialization parameters for 784-512-256-10 MLP model
num_classes = Y_train.shape[1]
num_features = X_train.shape[1]
num_output = Y_train.shape[1]
num_layers_0 = 200
num_layers_1 = 200
num_layers_2 = 200
starter_learning_rate = 0.001
regularizer_rate = 0.1

# Placeholders for the input data
input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
input_Y = tf.placeholder('float32',shape = (None,num_classes),name='input_Y')



## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
weights_0 = tf.Variable(tf.random_normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
bias_0 = tf.Variable(tf.random_normal([num_layers_0]))
weights_1 = tf.Variable(tf.random_normal([num_layers_0,num_layers_1], stddev=(1/tf.sqrt(float(num_layers_0)))))
bias_1 = tf.Variable(tf.random_normal([num_layers_1]))
weights_2 = tf.Variable(tf.random_normal([num_layers_1,num_layers_2], stddev=(1/tf.sqrt(float(num_layers_1)))))
bias_2 = tf.Variable(tf.random_normal([num_layers_2]))
weights_3 = tf.Variable(tf.random_normal([num_layers_2,num_output], stddev=(1/tf.sqrt(float(num_layers_2)))))
bias_3 = tf.Variable(tf.random_normal([num_output]))

# for dropout layer
keep_prob = tf.placeholder(tf.float32)
# Initializing weigths and biases -- with dropout
h0 = tf.matmul(input_X,weights_0)+bias_0
batch_mean0, batch_var0 = tf.nn.moments(h0,[0])
hidden_output_0 = tf.nn.relu(tf.nn.batch_normalization(h0, batch_mean0, batch_var0, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_0_0 = tf.nn.dropout(hidden_output_0, keep_prob)
h1 = tf.matmul(hidden_output_0,weights_1)+bias_1
batch_mean1, batch_var1 = tf.nn.moments(h1,[0])
hidden_output_1 = tf.nn.relu(tf.nn.batch_normalization(h1, batch_mean1, batch_var1, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_1_1 = tf.nn.dropout(hidden_output_1, keep_prob)
h2 = tf.matmul(hidden_output_1,weights_2)+bias_2
batch_mean2, batch_var2 = tf.nn.moments(h2,[0])
hidden_output_2 = tf.nn.relu(tf.nn.batch_normalization(h2, batch_mean2, batch_var2, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_2_2 = tf.nn.dropout(hidden_output_2, keep_prob)
predicted_Y = tf.sigmoid(tf.matmul(hidden_output_2_2,weights_3) + bias_3)



# ## Initializing weigths and biases -- withOUT dropout
# hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0)+bias_0)
# hidden_output_1 = tf.nn.relu(tf.matmul(hidden_output_0,weights_1)+bias_1)
# hidden_output_2 = tf.nn.relu(tf.matmul(hidden_output_1,weights_2)+bias_2)
# predicted_Y = tf.sigmoid(tf.matmul(hidden_output_2,weights_3) + bias_3)

## Defining the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predicted_Y,labels=input_Y)) \
        + regularizer_rate*(tf.reduce_sum(tf.square(bias_0)) + tf.reduce_sum(tf.square(bias_1)) + tf.reduce_sum(tf.square(bias_2)))

## Variable learning rate
learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
## Adam optimzer for finding the right weight
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,weights_2,weights_3,
                                                                         bias_0,bias_1,bias_2,bias_3])
grads_wrt_input_tensor = tf.gradients(loss,input_X)[0]
# preoptimizer = tf.train.AdamOptimizer(learning_rate)
# # grads = preoptimizer.compute_gradients(input_Y)
# grads = preoptimizer.compute_gradients(loss)
# grad_placeholder = [(tf.placeholder("float", shape=grad[1].get_shape()), grad[1]) for grad in grads]
# optimizer = preoptimizer.minimize(loss,var_list=[weights_0,weights_1,weights_2,weights_3,bias_0,bias_1,bias_2,bias_3])

## Metrics definition
correct_prediction = tf.equal(tf.argmax(Y_train,1), tf.argmax(predicted_Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Training parameters
batch_size = 128
epochs=14
dropout_prob = 0.1
training_accuracy = []
training_loss = []
testing_accuracy = []
s.run(tf.global_variables_initializer())
for epoch in range(epochs):    
    arr = np.arange(X_train.shape[0])
    np.random.shuffle(arr)
    for index in range(0,X_train.shape[0],batch_size):
        _, grads_wrt_input = s.run([optimizer, grads_wrt_input_tensor], {input_X: X_train[arr[index:index+batch_size]],
                          input_Y: Y_train[arr[index:index+batch_size]],
                        keep_prob:dropout_prob})
        
#         vars_with_grads = s.run(grads, feed_dict={input_X: X_train[arr[index:index+batch_size]],
#                           input_Y: Y_train[arr[index:index+batch_size]],
#                         keep_prob:dropout_prob})
    
    training_accuracy.append(s.run(accuracy, feed_dict= {input_X:X_train, 
                                                         input_Y: Y_train,keep_prob:1}))
    training_loss.append(s.run(loss, {input_X: X_train, 
                                      input_Y: Y_train,keep_prob:1}))
    
    ## Evaluation of model
    testing_accuracy.append(accuracy_score(Y_test.argmax(1), 
                            s.run(predicted_Y, {input_X: X_test,keep_prob:1}).argmax(1)))
#     testing_accuracy.append(accuracy_score(Y_test.argmax(1), 
#                             s.run(predicted_Y, {input_X: X_test}).argmax(1)))
    print("Epoch:{0} | Train loss: {1:.2f} | Train acc: {2:.3f} | Test acc:{3:.3f}".format(epoch,
                                                                    training_loss[epoch],
                                                                    training_accuracy[epoch],
                                                                   testing_accuracy[epoch]))
# grad_vals = s.run([grad[0] for grad in grads])
# https://r2rt.com/implementing-batch-normalization-in-tensorflow.html

##########################################################################################################
################################## FGSM #############################################################
##########################################################################################################

prop_random = 0.2 # what proportion of the training set do you want to apply perturbations to (!= sampling!!!)
# Might not be wise to shuffle the dataset, given that we kept X and Y separate -- let's just generate random index to start from
import random
# last_index_to_sample_from = int(X_train.shape[0] - X_train.shape[0]*prop_random)
# start_index = random.randint(0,last_index_to_sample_from)
# end_index = start_index + int(X_train.shape[0]*prop_random)
# print("Sample index range: ", (start_index, end_index))

ind = np.arange(int(X_train.shape[0]))
np.random.shuffle(ind)#[0:int(X_train.shape[0]*prop_random)]
ind = ind[0:int(X_train.shape[0]*prop_random)]

# sample from training set, matrix addition
# sample_X_train = X_train[start_index,end_index] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
# sample_Y_train = Y_train[start_index,end_index]

sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
sample_Y_train = Y_train[[ind]]

import keras.backend as K
# from attack_utils import gen_grad

def fgsm(x, grad, eps=0.3, clipping=True):
    """
    FGSM attack.
    """
    # signed gradient
    normed_grad = K.sign(grad).eval()

    # Multiply by constant epsilon
    scaled_grad = eps * normed_grad

    # Add perturbation to original example to obtain adversarial example
    adv_x = K.stop_gradient(x + scaled_grad)

    if clipping:
        adv_x = K.clip(adv_x, 0, 1)
    return adv_x

# x_adv = fgsm(sample_X_train, grads_wrt_input[-1], eps=16/255, clipping=True).eval()

##########################################################################################################
################################## Adversarial Training #############################################################
##########################################################################################################

# execute adversarial training --> compile into single pipeline to be turned into SOLID-able pipeline component

# original model

# https://towardsdatascience.com/multi-layer-perceptron-using-tensorflow-9f3e218a4809

print("With Dropout & Batch Normalization & Adversarial Training")

import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score

from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
# Batch normalization implementation
# from https://github.com/tensorflow/tensorflow/issues/1122
def batch_norm_layer(inputT, is_training=True, scope=None):
    # Note: is_training is tf.placeholder(tf.bool) type
    return tf.cond(is_training,
                    lambda: batch_norm(inputT, is_training=True,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9, scope=scope),
                    lambda: batch_norm(inputT, is_training=False,
                    center=True, scale=True, activation_fn=tf.nn.relu, decay=0.9,
                    scope=scope, reuse = True))


s = tf.InteractiveSession()

prop_random = 0.2 # what proportion of the training set do you want to apply perturbations to (!= sampling!!!)
adv_placeholders = tf.zeros([int(Y_train.shape[0]*prop_random) * Y_train.shape[1]]).eval().reshape(int(Y_train.shape[0]*prop_random), Y_train.shape[1])
modified_Y_train = np.array([list(Y_train)+list(adv_placeholders)][0])

## Defining various initialization parameters for 784-512-256-10 MLP model
num_classes = Y_train.shape[1]
num_features = X_train.shape[1]
num_output = Y_train.shape[1]
num_layers_0 = 200
num_layers_1 = 200
num_layers_2 = 200
starter_learning_rate = 0.001
regularizer_rate = 0.1

# Placeholders for the input data
input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
input_Y = tf.placeholder('float32',shape = (None,num_classes),name='input_Y')



## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
weights_0 = tf.Variable(tf.random_normal([num_features,num_layers_0], stddev=(1/tf.sqrt(float(num_features)))))
bias_0 = tf.Variable(tf.random_normal([num_layers_0]))
weights_1 = tf.Variable(tf.random_normal([num_layers_0,num_layers_1], stddev=(1/tf.sqrt(float(num_layers_0)))))
bias_1 = tf.Variable(tf.random_normal([num_layers_1]))
weights_2 = tf.Variable(tf.random_normal([num_layers_1,num_layers_2], stddev=(1/tf.sqrt(float(num_layers_1)))))
bias_2 = tf.Variable(tf.random_normal([num_layers_2]))
weights_3 = tf.Variable(tf.random_normal([num_layers_2,num_output], stddev=(1/tf.sqrt(float(num_layers_2)))))
bias_3 = tf.Variable(tf.random_normal([num_output]))

# for dropout layer
keep_prob = tf.placeholder(tf.float32)
# Initializing weigths and biases -- with dropout
h0 = tf.matmul(input_X,weights_0)+bias_0
batch_mean0, batch_var0 = tf.nn.moments(h0,[0])
hidden_output_0 = tf.nn.relu(tf.nn.batch_normalization(h0, batch_mean0, batch_var0, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_0_0 = tf.nn.dropout(hidden_output_0, keep_prob)
h1 = tf.matmul(hidden_output_0,weights_1)+bias_1
batch_mean1, batch_var1 = tf.nn.moments(h1,[0])
hidden_output_1 = tf.nn.relu(tf.nn.batch_normalization(h1, batch_mean1, batch_var1, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_1_1 = tf.nn.dropout(hidden_output_1, keep_prob)
h2 = tf.matmul(hidden_output_1,weights_2)+bias_2
batch_mean2, batch_var2 = tf.nn.moments(h2,[0])
hidden_output_2 = tf.nn.relu(tf.nn.batch_normalization(h2, batch_mean2, batch_var2, tf.Variable(tf.zeros([200])), tf.Variable(tf.ones([200])), 1e-3))
hidden_output_2_2 = tf.nn.dropout(hidden_output_2, keep_prob)
predicted_Y = tf.sigmoid(tf.matmul(hidden_output_2_2,weights_3) + bias_3)



# ## Initializing weigths and biases -- withOUT dropout
# hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0)+bias_0)
# hidden_output_1 = tf.nn.relu(tf.matmul(hidden_output_0,weights_1)+bias_1)
# hidden_output_2 = tf.nn.relu(tf.matmul(hidden_output_1,weights_2)+bias_2)
# predicted_Y = tf.sigmoid(tf.matmul(hidden_output_2,weights_3) + bias_3)

## Defining the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predicted_Y,labels=input_Y)) \
        + regularizer_rate*(tf.reduce_sum(tf.square(bias_0)) + tf.reduce_sum(tf.square(bias_1)) + tf.reduce_sum(tf.square(bias_2)))

## Variable learning rate
learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)
## Adam optimzer for finding the right weight
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss,var_list=[weights_0,weights_1,weights_2,weights_3,
                                                                         bias_0,bias_1,bias_2,bias_3])
# grads_wrt_input_tensor = tf.gradients(loss,input_X)[0]
# preoptimizer = tf.train.AdamOptimizer(learning_rate)
# # grads = preoptimizer.compute_gradients(input_Y)
# grads = preoptimizer.compute_gradients(loss)
# grad_placeholder = [(tf.placeholder("float", shape=grad[1].get_shape()), grad[1]) for grad in grads]
# optimizer = preoptimizer.minimize(loss,var_list=[weights_0,weights_1,weights_2,weights_3,bias_0,bias_1,bias_2,bias_3])

## Metrics definition
correct_prediction = tf.equal(tf.argmax(modified_Y_train,1), tf.argmax(predicted_Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Training parameters
batch_size = 128
epochs=14
dropout_prob = 0.1
training_accuracy = []
training_loss = []
testing_accuracy = []
s.run(tf.global_variables_initializer())
for epoch in range(epochs):    
    
    # regenerate training set
    # Might not be wise to shuffle the dataset, given that we kept X and Y separate -- let's just generate random index to start from
    import random

    ind = np.arange(int(X_train.shape[0]))
    np.random.shuffle(ind)#[0:int(X_train.shape[0]*prop_random)]
    ind = ind[0:int(X_train.shape[0]*prop_random)]

    sample_X_train = X_train[[ind]] # X_train[[start_index,end_index]] ; this would let us take specific indices, so we can randomize the order, ensure each iteration is taking precisely unique samples
    sample_Y_train = Y_train[[ind]]
    x_adv = fgsm(sample_X_train, grads_wrt_input[-1], eps=16/255, clipping=True).eval()
    adv_combined_X_train = np.array([list(X_train)+list(x_adv)][0])
    adv_combined_Y_train = np.array([list(Y_train)+list(sample_Y_train)][0])
    
    
    arr = np.arange(adv_combined_X_train.shape[0])
    np.random.shuffle(arr)
    for index in range(0,adv_combined_X_train.shape[0],batch_size):
        s.run(optimizer, {input_X: adv_combined_X_train[arr[index:index+batch_size]],
                          input_Y: adv_combined_Y_train[arr[index:index+batch_size]],
                        keep_prob:dropout_prob})
        
#         vars_with_grads = s.run(grads, feed_dict={input_X: X_train[arr[index:index+batch_size]],
#                           input_Y: Y_train[arr[index:index+batch_size]],
#                         keep_prob:dropout_prob})
    
    training_accuracy.append(s.run(accuracy, feed_dict= {input_X:adv_combined_X_train, 
                                                         input_Y: adv_combined_Y_train,keep_prob:1}))
    training_loss.append(s.run(loss, {input_X: adv_combined_X_train, 
                                      input_Y: adv_combined_Y_train,keep_prob:1}))
    
    ## Evaluation of model
    testing_accuracy.append(accuracy_score(Y_test.argmax(1), 
                            s.run(predicted_Y, {input_X: X_test,keep_prob:1}).argmax(1)))
#     testing_accuracy.append(accuracy_score(Y_test.argmax(1), 
#                             s.run(predicted_Y, {input_X: X_test}).argmax(1)))
    print("Epoch:{0} | Train loss: {1:.2f} | Train acc: {2:.3f} | Test acc:{3:.3f}".format(epoch,
                                                                    training_loss[epoch],
                                                                    training_accuracy[epoch],
                                                                   testing_accuracy[epoch]))
# grad_vals = s.run([grad[0] for grad in grads])
# https://r2rt.com/implementing-batch-normalization-in-tensorflow.html

With Dropout & Batch Normalization


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch:0 | Train loss: 35.72 | Train acc: 0.948 | Test acc:0.950
Epoch:1 | Train loss: 20.09 | Train acc: 0.966 | Test acc:0.962
Epoch:2 | Train loss: 11.50 | Train acc: 0.973 | Test acc:0.967
Epoch:3 | Train loss: 6.77 | Train acc: 0.979 | Test acc:0.971
Epoch:4 | Train loss: 4.20 | Train acc: 0.983 | Test acc:0.974
Epoch:5 | Train loss: 2.83 | Train acc: 0.985 | Test acc:0.976
Epoch:6 | Train loss: 2.13 | Train acc: 0.986 | Test acc:0.975
Epoch:7 | Train loss: 1.79 | Train acc: 0.987 | Test acc:0.976
Epoch:8 | Train loss: 1.62 | Train acc: 0.988 | Test acc:0.977
Epoch:9 | Train loss: 1.54 | Train acc: 0.990 | Test acc:0.980
Epoch:10 | Train loss: 1.51 | Train acc: 0.991 | Test acc:0.979
Epoch:11 | Train loss: 1.49 | Train acc: 0.991 | Test acc:0.978
Epoch:12 | Train loss: 1.48 | Train acc: 0.991 | Test acc:0.978
Epoch:13 | Train loss: 1.47 | Train acc: 0.991 | Test acc:0.980
With Dropout & Batch Normalization & Adversarial Training


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:155: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:156: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interac

Epoch:0 | Train loss: 29.80 | Train acc: 0.811 | Test acc:0.952
Epoch:1 | Train loss: 15.07 | Train acc: 0.823 | Test acc:0.964
Epoch:2 | Train loss: 7.85 | Train acc: 0.831 | Test acc:0.969
Epoch:3 | Train loss: 4.38 | Train acc: 0.835 | Test acc:0.973
Epoch:4 | Train loss: 2.76 | Train acc: 0.837 | Test acc:0.976
Epoch:5 | Train loss: 2.03 | Train acc: 0.837 | Test acc:0.977
Epoch:6 | Train loss: 1.72 | Train acc: 0.840 | Test acc:0.978
Epoch:7 | Train loss: 1.58 | Train acc: 0.840 | Test acc:0.977
Epoch:8 | Train loss: 1.52 | Train acc: 0.842 | Test acc:0.979
Epoch:9 | Train loss: 1.49 | Train acc: 0.843 | Test acc:0.979
Epoch:10 | Train loss: 1.48 | Train acc: 0.841 | Test acc:0.979
Epoch:11 | Train loss: 1.47 | Train acc: 0.844 | Test acc:0.978
Epoch:12 | Train loss: 1.47 | Train acc: 0.844 | Test acc:0.979
Epoch:13 | Train loss: 1.47 | Train acc: 0.843 | Test acc:0.979


In [ ]:
# https://github.com/ftramer/ensemble-adv-training/blob/master/train_adv.py
# https://github.com/ftramer/ensemble-adv-training/blob/master/fgs.py